In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torchvision
import pathlib
import glob
from PIL import Image
from torch.optim import Adam
import torch.nn as nn
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [5]:
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
    [0.5,0.5,0.5]


    )
])

In [6]:
train_path = '/content/LeafDisease_2/dataset/train'
test_path = '/content/LeafDisease_2/dataset/test'

train_loader = DataLoader(torchvision.datasets.ImageFolder(train_path,transform = transformer,),
batch_size = 64,shuffle = True)

test_loader = DataLoader(torchvision.datasets.ImageFolder(test_path,transform = transformer,),
batch_size = 32,shuffle = True)


In [7]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in  root.iterdir()
])

In [8]:
print(classes)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___Healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust', 'Corn_(maize)___Healthy', 'Corn_(maize)___Northern_Leaf_Blight', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Healthy', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Potato___Early_blight', 'Potato___Healthy', 'Potato___Late_blight', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Healthy', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus']


In [9]:
len(classes)

25

In [38]:
from torchvision.models.resnet import resnet18
model = resnet18(pretrained = True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features,out_features = 25 )
model = model.to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
optimizer = Adam(model.parameters(),lr = 0.001,weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss()

In [17]:
train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))

In [18]:
print(train_count,test_count)

19 8


In [40]:
num_epochs = 8

In [41]:
best_accuracy = 0.0

#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for  i,(images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'Besst_checkpoint.model')
        best_accuracy=test_accuracy


Epoch: 0 Train Loss: tensor(67.5569) Train Accuracy: 0.631578947368421 Test Accuracy: 0.75
Epoch: 1 Train Loss: tensor(67.5709) Train Accuracy: 0.7368421052631579 Test Accuracy: 0.75
Epoch: 2 Train Loss: tensor(67.6062) Train Accuracy: 0.8421052631578947 Test Accuracy: 0.75
Epoch: 3 Train Loss: tensor(67.4078) Train Accuracy: 0.8947368421052632 Test Accuracy: 1.0
Epoch: 4 Train Loss: tensor(67.6866) Train Accuracy: 0.7894736842105263 Test Accuracy: 0.875
Epoch: 5 Train Loss: tensor(67.4175) Train Accuracy: 0.8947368421052632 Test Accuracy: 0.875
Epoch: 6 Train Loss: tensor(67.8406) Train Accuracy: 0.6842105263157895 Test Accuracy: 1.25
Epoch: 7 Train Loss: tensor(67.4119) Train Accuracy: 1.0 Test Accuracy: 1.0


In [42]:
checkpoint=torch.load('Besst_checkpoint.model')
model=model
model.load_state_dict(checkpoint)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [43]:
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [56]:
pred_path = '/content/LeafDisease_2/im_for_testing_purpose'

image_path = glob.glob(pred_path+'/*.JPG')


In [57]:
pred_dict = {}

for i in image_path:
  pred_dict[i[i.rfind('/')+1:]] = prediction(i,transformer)

In [58]:
pred_dict

{'p.earlyblight.JPG': 'Grape___Healthy',
 't.targetspot (2).JPG': 'Potato___Early_blight',
 'g.healthy.JPG': 'Apple___Apple_scab',
 'p.healthy.JPG': 'Apple___Cedar_apple_rust',
 'a.blackrot.JPG': 'Grape___Esca_(Black_Measles)',
 't.bacterialspot.JPG': 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 't.leafmold.JPG': 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'c.commonrust.JPG': 'Apple___Black_rot',
 'p.healthy (2).JPG': 'Apple___Cedar_apple_rust',
 't.yellocurls.JPG': 'Corn_(maize)___Healthy',
 'g.esca.JPG': 'Potato___Late_blight',
 't.lateblight.JPG': 'Grape___Esca_(Black_Measles)',
 'a.healthy.JPG': 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 't.mosaicvirus.JPG': 'Grape___Esca_(Black_Measles)',
 't.septleafmold.JPG': 'Grape___Black_rot',
 'g.leafblight.JPG': 'Grape___Esca_(Black_Measles)',
 't.spider.JPG': 'Grape___Healthy',
 'c.Cercosporaleaf_spotGray_leaf_spot.JPG': 'Corn_(maize)___Northern_Leaf_Blight',
 't.target1.JPG': 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 't.earl